In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

2024-04-28 16:06:26.619133: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 16:06:26.690908: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 16:06:26.692900: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 16:06:28.933410: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv("data/final_dataset/final_dataset.csv")
sample_text = df["text"][7]
sample_text

'Why do native americans hate it when it rains in april? because it brings mayflowers.'

In [3]:
def convert_to_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })


In [4]:
def predict(text):
    json_data = convert_to_json(text)
    
    endpoint = "http://localhost:8080/v1/models/humor_detection_model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]

    if prediction > 0.7:
        return  "Humor"
    else:
        return "non Humor"

In [5]:
predict(sample_text)

'Humor'